# Section ID Count and Event Data
- limited to WP:M pages

Section ID data was not limited to top-level (H2) sections during data capture, requiring post-capture processing for section ID click data.

Example of capture issue: https://en.wikipedia.org/wiki/Hepatitis#Signs_and_symptoms.
Clicks on links under "Acute hepatitis" were captured with section_id Acute_hepatitis, not Signs_and_symptoms.

Post-capture data augmentation: click event section_ids where mapped to parent H2 section headings. See [populate-section-table.ipynb](populate-section-table.ipynb) for extraction details.


In [1]:
# basic defaults, including study dates, common SQL exclusions and parquet files for anonymized data
%run -i 'data-defaults.py'

## Raw section data from captured events
- Raw total count of events (by all event types) for each section ID for WP:M pages only.
- Limited to >= 3000 events.
- shows the extent of the capture issue described above


In [2]:
# "raw" section data from captured events to show extent of the capture issue described above
pm_section_events_raw_query = """
SELECT section_id, action, count(*) count
FROM 
    citationusage 
WHERE page_id IN (SELECT page_id FROM ryanmax.population_wpm_pages_with_extlinks)
    AND wiki = 'enwiki'
    {}
    AND to_date(event_time) >= '{}'
    AND to_date(event_time) <= '{}'
    AND useragent_is_bot = FALSE
GROUP BY section_id, action
ORDER BY count desc
LIMIT 100
"""
pm_section_events_raw = spark.sql(
    pm_section_events_raw_query.format(
        event_exclusion_sql, start_date_string, end_date_string
    ))
pm_section_events_raw_pandas = pm_section_events_raw.toPandas()

In [3]:
section_pda_raw = pm_section_events_raw_pandas.copy()
# replace 'NaN' section_id with 'missing'
section_pda_raw.section_id.fillna(value='-- missing --', inplace=True)
# limit to counts of 1K or more
df_filtered_raw = section_pda_raw.query('count>3000').copy()
# set precision before pivot
df_filtered_raw['count'] = df_filtered_raw['count'].map(lambda x: '{0:.0f}'.format(x))
df_filtered_raw.pivot(index='section_id', columns='action', values='count')

action,extClick,fnClick,fnHover,upClick
section_id,,,,
-- missing --,143490,365209,523476,None
Adverse_effects,None,4252,6343,None
Background,None,None,3496,None
Cause,None,3821,7787,None
Causes,None,10930,18686,None
Classification,None,3029,6620,None
Criminal_charges,None,3025,None,None
Diagnosis,None,7353,14525,None
Downfall,None,None,3523,None


## Count of WP:M pages by top-level (H2) section ID

In [4]:
# count of top-level (H2) section IDs for WP:M pages only
pm_sections_query = """
SELECT section_h2, count(distinct page_id) count
FROM
    ryanmax.population_wpm_sections 
GROUP BY section_h2
ORDER BY count desc, section_h2
"""
pm_sections = spark.sql(pm_sections_query)
pm_sections.toPandas().head(50)

,section_h2,count
0,References,30862
1,External_links,18460
2,See_also,13327
3,History,5666
4,Diagnosis,4654
5,Treatment,4263
6,Further_reading,3165
7,Signs_and_symptoms,2839
8,Causes,2149
9,Epidemiology,2066


## Total count of events (all types) for each top-level (H2) section ID for WP:M pages only
- Limited to sections with more than 1K fnClick events
- **missing** values are largely because section IDs were not recorded "if the section is the Main Section" as per [Schema:CitationUsage](https://meta.wikimedia.org/wiki/Schema:CitationUsage).

In [5]:
# Total count of events (by all event types) for each top-level (H2) section ID for WP:M pages only
pm_section_events_query = """
SELECT population_wpm_sections.section_h2, action, count(*) count, count(*)/{} AS daily_average
FROM 
    citationusage
    LEFT JOIN ryanmax.population_wpm_sections 
        ON 
        population_wpm_sections.page_id = citationusage.page_id 
        AND population_wpm_sections.section_id = citationusage.section_id
WHERE
    wiki = 'enwiki'
    AND citationusage.page_id IN (SELECT page_id FROM ryanmax.population_wpm_pages_with_extlinks)
    {}
    AND to_date(citationusage.event_time) >= '{}'
    AND to_date(citationusage.event_time) <= '{}'
    AND useragent_is_bot = FALSE
GROUP BY population_wpm_sections.section_h2, action
ORDER BY count desc
"""
pm_section_events = spark.sql(
    pm_section_events_query.format(
        days_in_study,
        event_exclusion_sql, start_date_string, end_date_string
    ))
pm_section_events_pandas = pm_section_events.toPandas()

In [6]:
section_pda = pm_section_events_pandas.copy()
# replace 'NaN' section_h2 with 'missing'
section_pda.section_h2.fillna(value='-- missing --', inplace=True)
pivot=section_pda.pivot(index='section_h2', columns='action', values='count').query('fnClick > 1000')
section_h2s=pivot.index
pivot

action,extClick,fnClick,fnHover,upClick
section_h2,,,,
-- missing --,144124.0,367446.0,527633.0,6.0
Adverse_effects,206.0,8017.0,14158.0,NaN
Applications,18.0,1187.0,2429.0,NaN
Background,53.0,1853.0,4216.0,1.0
Biography,60.0,1700.0,3864.0,NaN
Career,124.0,1801.0,3876.0,NaN
Cause,174.0,8959.0,21897.0,NaN
Causes,386.0,20480.0,39966.0,1.0
Characteristics,66.0,1624.0,3835.0,NaN


#### Daily averages of above

In [7]:
section_pda.pivot(index='section_h2', columns='action', values='daily_average').loc[section_h2s]

action,extClick,fnClick,fnHover,upClick
section_h2,,,,
-- missing --,4503.87500,11482.68750,16488.53125,0.18750
Adverse_effects,6.43750,250.53125,442.43750,NaN
Applications,0.56250,37.09375,75.90625,NaN
Background,1.65625,57.90625,131.75000,0.03125
Biography,1.87500,53.12500,120.75000,NaN
Career,3.87500,56.28125,121.12500,NaN
Cause,5.43750,279.96875,684.28125,NaN
Causes,12.06250,640.00000,1248.93750,0.03125
Characteristics,2.06250,50.75000,119.84375,NaN


In [8]:
# count of events by event type and access mode for each top-level (H2) section ID for WP:M pages only
pm_section_events_mode_query = """
SELECT population_wpm_sections.section_h2, action, mode, count(*) count
FROM 
    citationusage
    LEFT JOIN ryanmax.population_wpm_sections 
        ON 
        population_wpm_sections.page_id = citationusage.page_id 
        AND population_wpm_sections.section_id = citationusage.section_id
WHERE
    wiki = 'enwiki'
    AND citationusage.page_id IN (SELECT page_id FROM ryanmax.population_wpm_pages_with_extlinks)
    {}
    AND to_date(citationusage.event_time) >= '{}'
    AND to_date(citationusage.event_time) <= '{}'
    AND useragent_is_bot = FALSE
GROUP BY population_wpm_sections.section_h2, action, mode
ORDER BY count desc
"""
pm_section_events_mode = spark.sql(
    pm_section_events_mode_query.format(
        event_exclusion_sql, start_date_string, end_date_string
    ))
pm_section_events_mode_pandas = pm_section_events_mode.toPandas()

## Desktop event counts for each top-level (H2) section ID for WP:M pages only
- limits above apply here as well

In [9]:
desktop_pda = pm_section_events_mode_pandas.query('mode == "desktop"').copy()
# replace 'NaN' section_h2 with 'missing'
desktop_pda.section_h2.fillna(value='-- missing --', inplace=True)
desktop_pda.pivot(index='section_h2', columns='action', values='count').query('fnClick > 1000')

action,extClick,fnClick,fnHover,upClick
section_h2,,,,
-- missing --,65483.0,123684.0,485050.0,6.0
Adverse_effects,49.0,2487.0,13283.0,NaN
Background,25.0,1024.0,4057.0,NaN
Cause,96.0,3624.0,20839.0,NaN
Causes,207.0,7194.0,37401.0,NaN
Chemistry,9.0,1232.0,3941.0,NaN
Classification,377.0,1697.0,7789.0,1.0
Diagnosis,722.0,6724.0,32950.0,NaN
Epidemiology,33.0,5265.0,20627.0,NaN


## Mobile event counts for each top-level (H2) section ID for WP:M pages only
- limits above apply here as well

In [10]:
mobile_pda = pm_section_events_mode_pandas.query('mode == "mobile"').copy()
# replace 'NaN' section_h2 with 'missing'
mobile_pda.section_h2.fillna(value='-- missing --', inplace=True)
mobile_pda.pivot(index='section_h2', columns='action', values='count').query('fnClick > 1000')

action,extClick,fnClick,fnHover,upClick
section_h2,,,,
-- missing --,78641.0,243762.0,42583.0,NaN
Adverse_effects,157.0,5530.0,875.0,NaN
Career,38.0,1044.0,132.0,NaN
Cause,78.0,5335.0,1058.0,NaN
Causes,179.0,13286.0,2565.0,1.0
Characteristics,4.0,1009.0,204.0,NaN
Classification,1225.0,2074.0,444.0,1.0
Diagnosis,1114.0,8924.0,1777.0,NaN
Early_life,12.0,1090.0,193.0,NaN


## Total event counts for Adverse_effects section H2s for WP:M pages
 - 20 pages with most events

In [11]:
# event counts for pages with Adverse_effects section H2s
adverse_events_query = """
SELECT citationusage.page_id, population_page_titles_20190420.page_title, count(*) AS event_count
FROM 
    citationusage
    LEFT JOIN ryanmax.population_wpm_sections 
        ON 
        population_wpm_sections.page_id = citationusage.page_id 
        AND population_wpm_sections.section_id = citationusage.section_id
    LEFT JOIN ryanmax.population_page_titles_20190420
        ON
        population_page_titles_20190420.page_id = citationusage.page_id 
WHERE
    wiki = 'enwiki'
    AND population_wpm_sections.section_h2 = 'Adverse_effects'
    AND citationusage.page_id IN (SELECT page_id FROM ryanmax.population_wpm_pages_with_extlinks)
    {}
    AND to_date(citationusage.event_time) >= '{}'
    AND to_date(citationusage.event_time) <= '{}'
    AND useragent_is_bot = FALSE
GROUP BY citationusage.page_id, population_page_titles_20190420.page_title
ORDER BY event_count desc
"""
adverse_events = spark.sql(
    adverse_events_query.format(
        event_exclusion_sql, start_date_string, end_date_string
    ))
adverse_events_pandas = adverse_events.toPandas()

In [12]:
print('total events for Adverse_effects: ',adverse_events_pandas['event_count'].sum())
adverse_events_pandas.head(20)

total events for Adverse_effects:  22381


,page_id,page_title,event_count
0,1481886,Cannabis (drug),1198
1,10024,MDMA,1099
2,1245311,Finasteride,511
3,17537,Lysergic acid diethylamide,464
4,83406,Paracetamol,433
5,141915,Fentanyl,393
6,724730,Clonazepam,375
7,8718425,Circumcision,339
8,155627,Ibuprofen,332
9,178197,Statin,315
